<h1 style="text-align:center">BERT для решения задачи multi-label классификации текстов.</h1>

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("data.csv", index_col='ID')

In [3]:
data = data.loc[0:100]

In [4]:
data.head(2)

,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
ID,,,,,,,,
1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0


In [5]:
target_cols = [
    'Computer Science',
    'Physics',
    'Mathematics',
    'Statistics',
    'Quantitative Biology',
    'Quantitative Finance'
]

In [6]:
X_df = pd.DataFrame(
    data['ABSTRACT'].values,
    columns = ['text']
) 

X_df.head(3)

,text
0,Predictive models allow subject-specific inf...
1,Rotation invariance and translation invarian...
2,We introduce and develop the notion of spher...


In [7]:
def make_target_list(row):
    target_list = []
    for target in target_cols:
        target_list.append(row[target])
    return target_list

In [8]:
y_df = pd.DataFrame(data.apply(make_target_list, axis=1), columns=['target_list'])
y_df

,target_list
ID,
1,"[1, 0, 0, 0, 0, 0]"
2,"[1, 0, 0, 0, 0, 0]"
3,"[0, 0, 1, 0, 0, 0]"
4,"[0, 0, 1, 0, 0, 0]"
5,"[1, 0, 0, 1, 0, 0]"
...,...
96,"[1, 0, 0, 1, 0, 0]"
97,"[1, 0, 0, 1, 0, 0]"
98,"[0, 0, 1, 0, 0, 0]"


In [9]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X_df, y_df, test_size=0.2, random_state=42
)

In [10]:
y_train

,target_list
ID,
56,"[0, 0, 0, 0, 1, 0]"
89,"[0, 1, 0, 0, 0, 0]"
27,"[1, 0, 0, 0, 0, 0]"
43,"[0, 1, 0, 0, 0, 0]"
70,"[0, 0, 1, 0, 0, 0]"
...,...
61,"[1, 0, 0, 0, 0, 0]"
72,"[0, 1, 0, 0, 0, 0]"
15,"[0, 1, 0, 0, 0, 0]"


# Предобработка данных к нужному формату

In [11]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset

In [12]:
from transformers import BertModel, BertTokenizer

In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [14]:
import warnings
warnings.filterwarnings("ignore")

In [15]:
class TextDataset(Dataset):
    """Класс для преобразования датасета к нужному формату"""
    def __init__(self, X, y, tokenizer, max_len=512):
        self.tokenizer = tokenizer
        self.sentences = X['text'].tolist()
        self.labels = y['target_list'].tolist()
        self.max_len = max_len

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        text = self.sentences[idx]
        target_list = self.labels[idx]
        # токенизируем
        inputs = self.tokenizer.encode_plus(
            text=text, # наши данные
            text_pair=None, # это для задачи вопросно-ответной системы, т.е. не для нас
            add_special_tokens=True, # добавление спец-токенов, отвечающих за "начало предложения" [CLS] и "конец предложения" [SEP]
            max_length=self.max_len, # максимальная длина последовательности
            padding='max_length', # если в предложении меньше 64 токенов, то остальные заменяем на пустые
            truncation=True, # если в предложениее 64+ токенов, то мы просто обрезаем их
            return_token_type_ids=False, # это для задачи вопросно-ответной системы, т.е. не для нас
            return_attention_mask=True, # это говорит нашей модели, какие токены важны, а какие просто как padding или [CLS] и т.д.
            return_tensors='pt' # формат выдачи токенизатора, в нашем случае - torch тензор
        )

        # то что мы запихнем в модель
        return {
            'input_ids': inputs['input_ids'].flatten(), # это наши цифровые токены (т.е. для токена 'привет' будет какое-нибудь '105')
            'attention_mask': inputs['attention_mask'].flatten(), # это наши маски
            'labels': torch.tensor(target_list, dtype=torch.float)
        }

In [16]:
bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=False)
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [17]:
train_dataset = TextDataset(X_train, y_train, bert_tokenizer)
val_dataset = TextDataset(X_val, y_val, bert_tokenizer)

In [18]:
batch_size = 8

train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_data_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Создадим архитеутру нейронки для решения задачи

In [19]:
class BertMultiLabel(torch.nn.Module):
    def __init__(self, bert):
        super(BertMultiLabel, self).__init__()
        self.bert = bert
        self.dropout = torch.nn.Dropout(0.2)
        self.fc = torch.nn.Linear(768, 6)

    def forward(self, ids, mask):
        _, output = self.bert(ids, attention_mask=mask)
        output = self.dropout(output)
        output = self.fc(output)
        return output

In [20]:
custom_model = BertMultiLabel(bert_model)
custom_model.to(device)

BertMultiLabel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [21]:
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(params =  custom_model.parameters(), lr=1e-05)

## Обучение модели

In [22]:
import tqdm
from tqdm.auto import tqdm

def train_val(model, train_data_loader, val_data_loader, loss_fn, optimizer, device, num_epochs):
    for t in tqdm(range(num_epochs)):
        train_epoch_loss = []
        eval_epoch_loss = []
        
        print("Эпоха номер: ", t)

        # Обучение модели на текущей эпохе
        model.train()
        for train_data in tqdm(train_data_loader):
            input_ids = train_data['input_ids'].to(device) # токены
            attention_mask = train_data['attention_mask'].to(device) # маски
            labels = train_data['labels'].to(device) # класс

            outputs = model(input_ids, attention_mask) # результат модели
            #_, preds = torch.max(outputs.logits, dim=1)
            
            loss = loss_fn(outputs, labels) # считаем потерю
            train_epoch_loss.append(loss.item())


            # Выполним подсчёт новых градиентов
            loss.backward()
            # Выполним шаг градиентного спуска
            optimizer.step()
            # Обнулим сохраненные у оптимизатора значения градиентов
            # перед следующим шагом обучения
            optimizer.zero_grad()

        # Оценка модели на валидационных данных после обучения на текущей эпохе
        model.eval()
        for val_data in tqdm(val_data_loader):
            input_ids = val_data['input_ids'].to(device) # токены
            attention_mask = val_data['attention_mask'].to(device) # маски
            labels = val_data['labels'].to(device) # класс


            with torch.no_grad():
                outputs = model(input_ids, attention_mask) # результат модели
            
                loss = loss_fn(outputs, labels) # считаем потерю
                eval_epoch_loss.append(loss.item())

        # Выведем результаты прошедшей эпохи обучения
        print("Train loss: ", np.mean(train_epoch_loss))
        print("Eval loss: ", np.mean(eval_epoch_loss))
        
    return model

In [23]:
trained_custom_model = train_val(custom_model, train_data_loader, val_data_loader, loss_fn, optimizer, device, 1)

  0%|          | 0/1 [00:00<?, ?it/s]

Эпоха номер:  0


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Train loss:  0.6357570469379425
Eval loss:  0.6081543564796448


In [24]:
torch.save(trained_custom_model.state_dict(), "./model")

In [42]:
ex_text = "Hello world! My name is Maksim and I am ML engineer"
tokenized = bert_tokenizer.encode_plus(
            text=ex_text, # наши данные
            text_pair=None, # это для задачи вопросно-ответной системы, т.е. не для нас
            add_special_tokens=True, # добавление спец-токенов, отвечающих за "начало предложения" [CLS] и "конец предложения" [SEP]
            max_length=512, # максимальная длина последовательности
            padding='max_length', # если в предложении меньше 64 токенов, то остальные заменяем на пустые
            truncation=True, # если в предложениее 64+ токенов, то мы просто обрезаем их
            return_token_type_ids=False, # это для задачи вопросно-ответной системы, т.е. не для нас
            return_attention_mask=True, # это говорит нашей модели, какие токены важны, а какие просто как padding или [CLS] и т.д.
            return_tensors='pt' # формат выдачи токенизатора, в нашем случае - torch тензор
        )
trained_custom_model(tokenized['input_ids'], tokenized['attention_mask'])

tensor([[ 0.0213,  0.0105,  0.2872, -0.4165, -0.7102, -0.1550]],
       grad_fn=<AddmmBackward0>)

?????????????????